# CSCI 7000 / CMMID - Final Project
- **Author**: Corey Lynn Murphey
- **Professor**: Dan Larremore
- **Deadline**: 10 May 2023


## COVID-19 Aerosol Transmission Estimator, Python Edition

Based on the spreadsheet from: 
- Jimenez, J. L. & Peng, Z. COVID-19 Aerosol Transmission Estimator. https://tinyurl.com/covid-estimator (version : 21-Mar-22).

## Import Libraries

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

### Unit Conversions

In [2]:
sec_per_min = 60
min_per_hour = 60
L_per_m3  = 1000
C_to_Kelvin = 273.15 
ft_per_meter = 3.28084
ft_to_meter = .305 # 1 ft = .305 meters

# Functions -> to use to convert spreadsheet to notebook

In [3]:
def netEmissionRate_fx(infectedQuantaExhalationRate, exhalationMaskEfficiency, propPeopleWithMasks, I_0):
    netEmissRate = infectedQuantaExhalationRate * (1 - exhalationMaskEfficiency*propPeopleWithMasks)*I_0 # infectious doses (quanta) h-1
    return netEmissRate

def avgQuantaConcentration_fx(netEmissionRate, firstOrderLossRate, volumeRoom, eventDuration):
    avgQuantaConcentration = netEmissionRate/firstOrderLossRate/volumeRoom*(1-(1/firstOrderLossRate/eventDuration)*(1-np.exp(-firstOrderLossRate*eventDuration))) # infectious doses (quanta) m-3
    return avgQuantaConcentration

def quantaInhaledPerPerson_fx(avgQuantaConcentration, breathingRate, propPeopleWithMasks, inhalationMaskEfficiency, eventDuration):
    quantaInhaledPerPerson = avgQuantaConcentration*breathingRate*eventDuration*(1 - propPeopleWithMasks*inhalationMaskEfficiency) # infectious doses (quanta)
    return quantaInhaledPerPerson

def probabilityOfInfection(quantaInhaledPerPerson):
    #Conditional Results for A GIVEN PERSON & ONE EVENT (assuming number of infected above, typically 1)
    probOfInf = 1 - np.exp(-quantaInhaledPerPerson)
    return probOfInf

def newCases(probOfInf, S_0):
    # Conditional Results for ALL ATTENDEES & ONE EVENT (assuming number of infected above, typically 1
    numNewCases = probOfInf * S_0
    return numNewCases

# Airborne Infection Risk Parameters (From Peng et al., 2022)
def infectionRisk(relBreathingRateFactor, vocalEnhancement, exhalationMaskEfficiency, propPeopleWithMasks, inhalationMaskEfficiency, S_0, volumeRoom, firstOrderLossRate, eventDuration):
    infRiskParam = relBreathingRateFactor * vocalEnhancement * (1-exhalationMaskEfficiency*propPeopleWithMasks)*(1-propPeopleWithMasks*inhalationMaskEfficiency)*eventDuration*S_0/(firstOrderLossRate*volumeRoom)*(1-(1-np.exp(-firstOrderLossRate*eventDuration))/(firstOrderLossRate*eventDuration))
    return infRiskParam

def relInfectionRisk(infRiskParam, S_0):
    relInfRiskParam = infRiskParam/S_0
    return relInfRiskParam

In [4]:
def main(roomParams, envParams, 
                           occupants, eventParams, 
                           breathParams, enhancements, 
                           masking, ventilation, 
                           pathogenParams): 
    lengthRoom, widthRoom, heightRoom = roomParams
    
    floorArea = lengthRoom * widthRoom
    volumeRoom = lengthRoom * widthRoom * heightRoom
    
    #environment
    pressure, temperature, relativeHumidity, backgroundCO2 = envParams
    
    #occupants
    numPeople, I_0, s_0 = occupants 
    S_0 = (numPeople - I_0)*(1 - s_0) #number of susceptible people at time 0
    
    # densities
    density_AreaPerPerson = floorArea/(0.305**2)/numPeople #sq ft / person # TODO : Why hard coded? 
    density_PeoplePerArea = numPeople/floorArea
    density_VolumePerPerson = volumeRoom/numPeople
    
    #breathing
    breathingRate, basicBreathingRate, co2EmissionRate_person = breathParams
    relBreathingRateFactor = breathingRate/basicBreathingRate
    co2EmissionRate_total = co2EmissionRate_person*numPeople*(1/pressure)*(C_to_Kelvin+temperature)/C_to_Kelvin # L/s (@ at actual P & T of room)
    
    #enhancement
    quantaExhalationRate, variantEnhancement, vocalEnhancement = enhancements
    infectedQuantaExhalationRate = quantaExhalationRate * variantEnhancement * vocalEnhancement
    
    #masking
    exhalationMaskEfficiency, propPeopleWithMasks,inhalationMaskEfficiency = masking
    
    #event
    eventDuration_mins, eventRepetitions = eventParams
    eventDuration = eventDuration_mins/min_per_hour # hours

    #ventilation
    ventilationRate, virusDecayRate, depositionRate, additionalControlMeasureRate = ventilation
    
    firstOrderLossRate = ventilationRate + virusDecayRate + depositionRate + additionalControlMeasureRate
    ventilationPerPerson = (volumeRoom * (ventilationRate + additionalControlMeasureRate)/numPeople) # [m^3/hour/person] 
    ventilationPerPerson = ventilationPerPerson*L_per_m3/(sec_per_min * min_per_hour) # [L/s/person]

    # # pathogen, Not yet used
    # probInfective, hospitalizationRate, deathRate = pathogenParams
    
    # -------------- Outputs
    netEmissionRate = netEmissionRate_fx(infectedQuantaExhalationRate, exhalationMaskEfficiency, propPeopleWithMasks, I_0)
    avgQuantaConcentration = avgQuantaConcentration_fx(netEmissionRate, firstOrderLossRate, volumeRoom, eventDuration)
    quantaInhaledPerPerson = quantaInhaledPerPerson_fx(avgQuantaConcentration, breathingRate, propPeopleWithMasks, inhalationMaskEfficiency, eventDuration)

    probOfInf = probabilityOfInfection(quantaInhaledPerPerson)
    numNewCases = newCases(probOfInf, S_0)
    infRiskParam = infectionRisk(relBreathingRateFactor, vocalEnhancement, exhalationMaskEfficiency, propPeopleWithMasks, inhalationMaskEfficiency, S_0, volumeRoom, firstOrderLossRate, eventDuration)
    relInfRiskParam = relInfectionRisk(infRiskParam, S_0)
    
    return netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam


def printTheResults(netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam):
    print("The net emission rate is {}".format(netEmissionRate))
    print("The average quanta concentration is {}".format(avgQuantaConcentration))
    print("The quanta inhalation per person is {}".format(quantaInhaledPerPerson))
    print("----------")
    print("The probability of infection is {}".format( probOfInf))
    print("The number of new cases is {}".format(numNewCases))
    print("The infection risk is {} [h^2*person/m^3]".format(infRiskParam))
    print("The relative infection risk  is {}[h^2/m^3]".format(relInfRiskParam))

    

# Master Example - Choir
Based on the Skagit Valley Choir

## Parameters

### Room Parameters

In [5]:
lengthRoom = 9.2 # m
widthRoom = 18.3 # m
heightRoom = 4.8 # m 

floorArea = lengthRoom * widthRoom
volumeRoom = lengthRoom * widthRoom * heightRoom

### Environmental Parameters

In [6]:
pressure = 0.95 #atm, Used only for CO2 calculation
temperature = 20 # C, Use web converter if needed for F --> C. Used for CO2 calculation, eventually for survival rate of virus
relativeHumidity = 50 # percentage, Not yet used, but may eventually be used for survival rate of virus
backgroundCO2 = 415 # ppm 

### Occupants

In [7]:
numPeople = 61

# N.B. Modified variable names to correspond with SIR models
I_0 = 1 # number of infected people at time 0
s_0 = 0 # fraction of people susceptible at time 0 
S_0 = (numPeople - I_0)*(1 - s_0) #number of susceptible people at time 0

# display(S_0)

density_AreaPerPerson = floorArea/(ft_to_meter**2)/numPeople #sq ft / person 
density_PeoplePerArea = numPeople/floorArea
density_VolumePerPerson = volumeRoom/numPeople

# display(density_AreaPerPerson)
# display(density_PeoplePerArea)
# display(density_VolumePerPerson)

#### Breath parameters

In [8]:
breathingRate = 0.026 * min_per_hour #See Readme sheet  - varies a lot with activity level
# as best as I can tell, this comes from a moderate intensity breath rate for people ages 16 - 21
# but I'm not really sure where this number comes from 

basicBreathingRate = 0.0048 * min_per_hour # m3 / h, for a sedentary person in the 41 - 51 age groups (ASSUMPTION)
relBreathingRateFactor = breathingRate/basicBreathingRate

co2EmissionRate_person = 0.0091 # L/s (@ 273 K and 1 atm), From tables in Readme page. This does not affect infection calculation, only use of CO2 as indicator, could ignore
# could be completely incorrect for singing
# if 41-51 age range still applies, this means that the people are doin about 2-3 met of physical activity
co2EmissionRate_total = co2EmissionRate_person*numPeople*(1/pressure)*(C_to_Kelvin+temperature)/C_to_Kelvin # L/s (@ at actual P & T of room)

# display(relBreathingRateFactor)
# display(co2EmissionRate_total)

In [9]:
quantaExhalationRate = 18.6 # infectious doses (quanta) h-1, this is very uncertain
# The 5th and 95th percentiles of the Ep0 values are 8.4 and 48.1 quanta h−1, respectively.(Peng 2022)
variantEnhancement = 1 # Q. enhancement due to variants
vocalEnhancement = 52.15 # Q enhancement due to vocal. & act., Dimensionless (ratio to breathing)
# this value was not cited so I don't know where this came from

infectedQuantaExhalationRate = quantaExhalationRate * variantEnhancement * vocalEnhancement

# display(infectedQuantaExhalationRate)

##### Masking (none in this experiment)

In [10]:
exhalationMaskEfficiency = 0
propPeopleWithMasks = 0
inhalationMaskEfficiency = 0 

### Event Parameters

In [11]:
eventDuration_mins = 150
eventDuration = eventDuration_mins/min_per_hour # hours
eventRepetitions = 1 # if multiple meetings

# ventilation,  ventilation rate of 1 h-1 does not mean that 100% of the air is replaced in 1 h.
ventilationRate = 0.7 # h^-1, this may be low
virusDecayRate = 0.62 # h^-1, from literature. Uncertain parameter
depositionRate = 0.3 # h^-1,  Buonnano et al. (2020), Miller et al. (2020). Could vary 0.24-1.5 h-1, depending on particle size range
additionalControlMeasureRate = 0 # h^-1, E.g. filtering of recirc. air, HEPA air cleaner, UV disinfection, etc. 

# Loss Rate
firstOrderLossRate = ventilationRate + virusDecayRate + depositionRate + additionalControlMeasureRate

In [12]:
ventilationPerPerson = (volumeRoom * (ventilationRate + additionalControlMeasureRate)/numPeople) # [m^3/hour/person] 
ventilationPerPerson = ventilationPerPerson*L_per_m3/(sec_per_min * min_per_hour) # [L/s/person]

# display(ventilationPerPerson)

### Pathogen Parameters

For now, for COVID-19...

In [13]:
probInfective =7*2/129200 # %, Very important parameter, specific for each region and time period. For ABSOLUTE results (prob. given prevalence of disease in the population). See Readme sheet
# this is highly dependent on phase of epidemic and region 

hospitalizationRate = 20/100
deathRate = 4/100

# for this example, this is probably an artificially tuned

## Model Outputs

In [14]:
roomParams = lengthRoom, widthRoom, heightRoom
envParams = pressure, temperature, relativeHumidity, backgroundCO2 
occupants = numPeople, I_0, s_0
eventParams = eventDuration_mins, eventRepetitions
breathParams = breathingRate, basicBreathingRate, co2EmissionRate_person 
enhancements = quantaExhalationRate, variantEnhancement, vocalEnhancement
masking= exhalationMaskEfficiency, propPeopleWithMasks,inhalationMaskEfficiency
ventilation = ventilationRate, virusDecayRate, depositionRate, additionalControlMeasureRate 
pathogenParams = probInfective, hospitalizationRate, deathRate 

choirParams = [volumeRoom, numPeople, I_0, eventDuration_mins, eventRepetitions, propPeopleWithMasks, firstOrderLossRate]
netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam = main(roomParams, envParams, occupants, eventParams, breathParams, enhancements, masking, ventilation, pathogenParams)

choirRes = [netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam]

printTheResults(netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam)

The net emission rate is 969.99
The average quanta concentration is 0.5611650901148105
The quanta inhalation per person is 2.188543851447761
----------
The probability of infection is 0.8879201652612078
The number of new cases is 53.27520991567246
The infection risk is 24.513259984853953 [h^2*person/m^3]
The relative infection risk  is 0.4085543330808992[h^2/m^3]


# Classroom Example

### Room Parameters

In [15]:
lengthRoom = 7.6 # m
widthRoom = 6.1 # m
heightRoom = 3.1 # m 

floorArea = lengthRoom * widthRoom
volumeRoom = lengthRoom * widthRoom * heightRoom

### Environmental Parameters

In [16]:
pressure = 0.95 #atm, Used only for CO2 calculation
temperature = 20 # C, Use web converter if needed for F --> C. Used for CO2 calculation, eventually for survival rate of virus
relativeHumidity = 50 # percentage, Not yet used, but may eventually be used for survival rate of virus
backgroundCO2 = 415 # ppm 

### Occupants

In [17]:
numPeople = 10

# N.B. Modified variable names to correspond with SIR models
I_0 = 1 # number of infected people at time 0
s_0 = 0 # fraction of people susceptible at time 0 
S_0 = (numPeople - I_0)*(1 - s_0) #number of susceptible people at time 0

# display(S_0)

density_AreaPerPerson = floorArea/(ft_to_meter**2)/numPeople #sq ft / person 
density_PeoplePerArea = numPeople/floorArea
density_VolumePerPerson = volumeRoom/numPeople

# display(density_AreaPerPerson)
# display(density_PeoplePerArea)
# display(density_VolumePerPerson)

#### Breath parameters

In [18]:
breathingRate = 0.0086 * min_per_hour #See Readme sheet  - varies a lot with activity level
# as best as I can tell, this comes from a moderate intensity breath rate for people ages 16 - 21
# but I'm not really sure where this number comes from 
# Why is this value higher than the choir? 

basicBreathingRate = 0.0048 * min_per_hour # m3 / h, for a sedentary person in the 41 - 51 age groups (ASSUMPTION)
relBreathingRateFactor = breathingRate/basicBreathingRate

co2EmissionRate_person = 0.005 # L/s (@ 273 K and 1 atm), From tables in Readme page. This does not affect infection calculation, only use of CO2 as indicator, could ignore
# could be completely incorrect for singing
co2EmissionRate_total = co2EmissionRate_person*numPeople*(1/pressure)*(C_to_Kelvin+temperature)/C_to_Kelvin # L/s (@ at actual P & T of room)

# display(relBreathingRateFactor)
# display(co2EmissionRate_total)

In [19]:
quantaExhalationRate = 18.6 # infectious doses (quanta) h-1, this is very uncertain
# The 5th and 95th percentiles of the Ep0 values are 8.4 and 48.1 quanta h−1, respectively.(Peng 2022)
variantEnhancement = 2.5 # Q. enhancement due to variants (omicron here)
vocalEnhancement = 1.3 # Q enhancement due to vocal. & act., Dimensionless (ratio to breathing)
# this value was not cited so I don't know where this came from

infectedQuantaExhalationRate = quantaExhalationRate * variantEnhancement * vocalEnhancement

# display(infectedQuantaExhalationRate)

##### Masking (none in this experiment)

In [20]:
exhalationMaskEfficiency = 0.5
propPeopleWithMasks = 1
inhalationMaskEfficiency = 0.3

### Event Parameters

In [21]:
eventDuration_mins = 50
eventDuration = eventDuration_mins/min_per_hour # hours
# eventRepetitions = 180 # number of classes met per school year
eventRepetitions = 1 

# ventilation,  ventilation rate of 1 h-1 does not mean that 100% of the air is replaced in 1 h.
ventilationRate = 3 # h^-1, this may be low
virusDecayRate = 0.62 # h^-1, from literature. Uncertain parameter
depositionRate = 0.3 # h^-1,  Buonnano et al. (2020), Miller et al. (2020). Could vary 0.24-1.5 h-1, depending on particle size range
additionalControlMeasureRate = 0 # h^-1, E.g. filtering of recirc. air, HEPA air cleaner, UV disinfection, etc. 

# Loss Rate
firstOrderLossRate = ventilationRate + virusDecayRate + depositionRate + additionalControlMeasureRate

In [22]:
ventilationPerPerson = (volumeRoom * (ventilationRate + additionalControlMeasureRate)/numPeople) # [m^3/hour/person] 
ventilationPerPerson = ventilationPerPerson*L_per_m3/(sec_per_min * min_per_hour) # [L/s/person]

# display(ventilationPerPerson)

### Pathogen Parameters

For now, for COVID-19, Omicron variant

In [23]:
probInfective =0.002 # %, Very important parameter, specific for each region and time period. For ABSOLUTE results (prob. given prevalence of disease in the population). See Readme sheet
# this is highly dependent on phase of epidemic and region 

hospitalizationRate = 10/100
deathRate = 1/100

# for this example, this is probably an artificially tuned

## Model Outputs

### Main (calculate baseline emissions)

In [24]:
roomParams = lengthRoom, widthRoom, heightRoom
envParams = pressure, temperature, relativeHumidity, backgroundCO2 
occupants = numPeople, I_0, s_0
eventParams = eventDuration_mins, eventRepetitions
breathParams = breathingRate, basicBreathingRate, co2EmissionRate_person 
enhancements = quantaExhalationRate, variantEnhancement, vocalEnhancement
masking= exhalationMaskEfficiency, propPeopleWithMasks,inhalationMaskEfficiency
ventilation = ventilationRate, virusDecayRate, depositionRate, additionalControlMeasureRate 
pathogenParams = probInfective, hospitalizationRate, deathRate 

classParams = [volumeRoom, numPeople, I_0, eventDuration_mins, eventRepetitions, propPeopleWithMasks, firstOrderLossRate]

netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam = main(roomParams, envParams, occupants, eventParams, breathParams, enhancements, masking, ventilation, pathogenParams)

classRes = [netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam]

printTheResults(netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam)

The net emission rate is 30.225
The average quanta concentration is 0.03785328236827028
The quanta inhalation per person is 0.011393837992849353
----------
The probability of infection is 0.011329174043938961
The number of new cases is 0.10196256639545065
The infection risk is 0.007657149188742846 [h^2*person/m^3]
The relative infection risk  is 0.0008507943543047606[h^2/m^3]


# Subway Example

### Room Parameters

In [25]:
lengthRoom = 13.6 # m
widthRoom = 3.1 # m
heightRoom = 3.7 # m 

floorArea = lengthRoom * widthRoom
volumeRoom = lengthRoom * widthRoom * heightRoom

### Environmental Parameters

In [26]:
pressure = 0.95 #atm, Used only for CO2 calculation
temperature = 20 # C, Use web converter if needed for F --> C. Used for CO2 calculation, eventually for survival rate of virus
relativeHumidity = 50 # percentage, Not yet used, but may eventually be used for survival rate of virus
backgroundCO2 = 415 # ppm 

### Occupants

In [27]:
numPeople = 35

# N.B. Modified variable names to correspond with SIR models
I_0 = 1 # number of infected people at time 0
# r_0 = .15
r_0 = 0
S_0 = (numPeople - I_0)*(1 - r_0) #number of susceptible people at time 0

# display(S_0)

density_AreaPerPerson = floorArea/(ft_to_meter**2)/numPeople #sq ft / person 
density_PeoplePerArea = numPeople/floorArea
density_VolumePerPerson = volumeRoom/numPeople

# display(density_AreaPerPerson)
# display(density_PeoplePerArea)
# display(density_VolumePerPerson)

#### Breath parameters

In [28]:
breathingRate = 0.007 * min_per_hour #See Readme sheet  - varies a lot with activity level
# as best as I can tell, this comes from a moderate intensity breath rate for people ages 16 - 21
# but I'm not really sure where this number comes from 
# Why is this value higher than the choir? 

basicBreathingRate = 0.0048 * min_per_hour # m3 / h, for a sedentary person in the 41 - 51 age groups (ASSUMPTION)
relBreathingRateFactor = breathingRate/basicBreathingRate

co2EmissionRate_person = 0.007 # L/s (@ 273 K and 1 atm), From tables in Readme page. This does not affect infection calculation, only use of CO2 as indicator, could ignore
# could be completely incorrect for singing
co2EmissionRate_total = co2EmissionRate_person*numPeople*(1/pressure)*(C_to_Kelvin+temperature)/C_to_Kelvin # L/s (@ at actual P & T of room)

# display(relBreathingRateFactor)
# display(co2EmissionRate_total)

In [29]:
quantaExhalationRate = 18.6 # infectious doses (quanta) h-1, this is very uncertain
# The 5th and 95th percentiles of the Ep0 values are 8.4 and 48.1 quanta h−1, respectively.(Peng 2022)
variantEnhancement = 2.5 # Q. enhancement due to variants (omicron here)
vocalEnhancement = 1 # Q enhancement due to vocal. & act., Dimensionless (ratio to breathing)
# this value was not cited so I don't know where this came from

infectedQuantaExhalationRate = quantaExhalationRate * variantEnhancement * vocalEnhancement

# display(infectedQuantaExhalationRate)

##### Masking (none in this experiment)

In [30]:
exhalationMaskEfficiency = 0.5
propPeopleWithMasks = 1
inhalationMaskEfficiency = 0.3

### Event Parameters

In [31]:
eventDuration_mins = 20
eventDuration = eventDuration_mins/min_per_hour # hours
# eventRepetitions = 60 # number of subway rides 
eventRepetitions = 1

# ventilation,  ventilation rate of 1 h-1 does not mean that 100% of the air is replaced in 1 h.
ventilationRate = 5.7 # h^-1, this may be low
virusDecayRate = 0.62 # h^-1, from literature. Uncertain parameter
depositionRate = 0.3 # h^-1,  Buonnano et al. (2020), Miller et al. (2020). Could vary 0.24-1.5 h-1, depending on particle size range
additionalControlMeasureRate = 3.6 # h^-1, E.g. filtering of recirc. air, HEPA air cleaner, UV disinfection, etc. 

# Loss Rate
firstOrderLossRate = ventilationRate + virusDecayRate + depositionRate + additionalControlMeasureRate

In [32]:
ventilationPerPerson = (volumeRoom * (ventilationRate + additionalControlMeasureRate)/numPeople) # [m^3/hour/person] 
ventilationPerPerson = ventilationPerPerson*L_per_m3/(sec_per_min * min_per_hour) # [L/s/person]

# display(ventilationPerPerson)

### Pathogen Parameters

For now, for COVID-19, Omicron variant

In [33]:
probInfective =0.001 # %, Very important parameter, specific for each region and time period. For ABSOLUTE results (prob. given prevalence of disease in the population). See Readme sheet
# this is highly dependent on phase of epidemic and region 

hospitalizationRate = 10/100
deathRate = 1/100

# for this example, this is probably an artificially tuned

## Model Outputs

### Main (calculate baseline emissions)

In [34]:
roomParams = lengthRoom, widthRoom, heightRoom
envParams = pressure, temperature, relativeHumidity, backgroundCO2 
occupants = numPeople, I_0, s_0
eventParams = eventDuration_mins, eventRepetitions
breathParams = breathingRate, basicBreathingRate, co2EmissionRate_person 
enhancements = quantaExhalationRate, variantEnhancement, vocalEnhancement
masking= exhalationMaskEfficiency, propPeopleWithMasks,inhalationMaskEfficiency
ventilation = ventilationRate, virusDecayRate, depositionRate, additionalControlMeasureRate 
pathogenParams = probInfective, hospitalizationRate, deathRate 

subwayParams = [volumeRoom, numPeople, I_0, eventDuration_mins, eventRepetitions, propPeopleWithMasks, firstOrderLossRate]

netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam = main(roomParams, envParams, occupants, eventParams, breathParams, enhancements, masking, ventilation, pathogenParams)

subwayRes = [netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam]

printTheResults(netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam)

The net emission rate is 23.25
The average quanta concentration is 0.010444738196364758
The quanta inhalation per person is 0.001023584343243746
----------
The probability of infection is 0.0010230606594833214
The number of new cases is 0.034784062422432926
The infection risk is 0.0025987057698840634 [h^2*person/m^3]
The relative infection risk  is 7.643252264364893e-05[h^2/m^3]


# Supermarket Example

### Room Parameters

In [35]:
lengthRoom = 24.4 # m
widthRoom = 15.3 # m
heightRoom = 5.5 # m 

floorArea = lengthRoom * widthRoom
volumeRoom = lengthRoom * widthRoom * heightRoom

### Environmental Parameters

In [36]:
pressure = 0.95 #atm, Used only for CO2 calculation
temperature = 20 # C, Use web converter if needed for F --> C. Used for CO2 calculation, eventually for survival rate of virus
relativeHumidity = 50 # percentage, Not yet used, but may eventually be used for survival rate of virus
backgroundCO2 = 415 # ppm 

### Occupants

In [37]:
numPeople = 75

# N.B. Modified variable names to correspond with SIR models
I_0 = 1 # number of infected people at time 0
# r_0 = 0.06 # fraction of people recovered at time 0 
r_0 = 0 
S_0 = (numPeople - I_0)*(1 - r_0) #number of susceptible people at time 0

# display(S_0)

density_AreaPerPerson = floorArea/(ft_to_meter**2)/numPeople #sq ft / person 
density_PeoplePerArea = numPeople/floorArea
density_VolumePerPerson = volumeRoom/numPeople

# display(density_AreaPerPerson)
# display(density_PeoplePerArea)
# display(density_VolumePerPerson)

#### Breath parameters

In [38]:
breathingRate = 0.012 * min_per_hour #See Readme sheet  - varies a lot with activity level
# as best as I can tell, this comes from a moderate intensity breath rate for people ages 16 - 21
# but I'm not really sure where this number comes from 
# Why is this value higher than the choir? 

basicBreathingRate = 0.0048 * min_per_hour # m3 / h, for a sedentary person in the 41 - 51 age groups (ASSUMPTION)
relBreathingRateFactor = breathingRate/basicBreathingRate

co2EmissionRate_person = 0.00675 # L/s (@ 273 K and 1 atm), From tables in Readme page. This does not affect infection calculation, only use of CO2 as indicator, could ignore
# could be completely incorrect for singing
co2EmissionRate_total = co2EmissionRate_person*numPeople*(1/pressure)*(C_to_Kelvin+temperature)/C_to_Kelvin # L/s (@ at actual P & T of room)

# display(relBreathingRateFactor)
# display(co2EmissionRate_total)

In [39]:
quantaExhalationRate = 18.6 # infectious doses (quanta) h-1, this is very uncertain
# The 5th and 95th percentiles of the Ep0 values are 8.4 and 48.1 quanta h−1, respectively.(Peng 2022)
variantEnhancement = 2.5 # Q. enhancement due to variants (omicron here)
vocalEnhancement = 5.0 # Q enhancement due to vocal. & act., Dimensionless (ratio to breathing)
# this value was not cited so I don't know where this came from

infectedQuantaExhalationRate = quantaExhalationRate * variantEnhancement * vocalEnhancement

# display(infectedQuantaExhalationRate)

##### Masking (none in this experiment)

In [40]:
exhalationMaskEfficiency = 0.5
propPeopleWithMasks = 1
inhalationMaskEfficiency = 0.3

### Event Parameters

In [41]:
eventDuration_mins = 480 # for a worker in the supermarket
eventDuration = eventDuration_mins/min_per_hour # hours
# eventRepetitions = 21 # number of supermarket met per month?
eventRepetitions = 1

# ventilation,  ventilation rate of 1 h-1 does not mean that 100% of the air is replaced in 1 h.
ventilationRate = 3 # h^-1, this may be low
virusDecayRate = 0.62 # h^-1, from literature. Uncertain parameter
depositionRate = 0.3 # h^-1,  Buonnano et al. (2020), Miller et al. (2020). Could vary 0.24-1.5 h-1, depending on particle size range
additionalControlMeasureRate = 0 # h^-1, E.g. filtering of recirc. air, HEPA air cleaner, UV disinfection, etc. 

# Loss Rate
firstOrderLossRate = ventilationRate + virusDecayRate + depositionRate + additionalControlMeasureRate

In [42]:
ventilationPerPerson = (volumeRoom * (ventilationRate + additionalControlMeasureRate)/numPeople) # [m^3/hour/person] 
ventilationPerPerson = ventilationPerPerson*L_per_m3/(sec_per_min * min_per_hour) # [L/s/person]

# display(ventilationPerPerson)

### Pathogen Parameters

For now, for COVID-19, Omicron variant

In [43]:
probInfective =0.001 # %, Very important parameter, specific for each region and time period. For ABSOLUTE results (prob. given prevalence of disease in the population). See Readme sheet
# this is highly dependent on phase of epidemic and region 

hospitalizationRate = 10/100
deathRate = 1/100

# for this example, this is probably an artificially tuned

## Model Outputs

### Main (calculate baseline emissions)

In [44]:
roomParams = lengthRoom, widthRoom, heightRoom
envParams = pressure, temperature, relativeHumidity, backgroundCO2 
occupants = numPeople, I_0, s_0
eventParams = eventDuration_mins, eventRepetitions
breathParams = breathingRate, basicBreathingRate, co2EmissionRate_person 
enhancements = quantaExhalationRate, variantEnhancement, vocalEnhancement
masking= exhalationMaskEfficiency, propPeopleWithMasks,inhalationMaskEfficiency
ventilation = ventilationRate, virusDecayRate, depositionRate, additionalControlMeasureRate 
pathogenParams = probInfective, hospitalizationRate, deathRate 

supermarketParams = [volumeRoom, numPeople, I_0, eventDuration_mins, eventRepetitions, propPeopleWithMasks, firstOrderLossRate]

netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam = main(roomParams, envParams, occupants, eventParams, breathParams, enhancements, masking, ventilation, pathogenParams)

supermarketRes = [netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam]

printTheResults(netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam)

The net emission rate is 116.25
The average quanta concentration is 0.013982623410689147
The quanta inhalation per person is 0.05637793759189863
----------
The probability of infection is 0.05481815137327117
The number of new cases is 4.056543201622067
The infection risk is 0.31152683555857974 [h^2*person/m^3]
The relative infection risk  is 0.00420982210214297[h^2/m^3]


# Stadium Example

### Room Parameters

In [45]:
lengthRoom = 183 # m
widthRoom = 91.5 # m
heightRoom = 15.3 # m 

floorArea = lengthRoom * widthRoom
volumeRoom = lengthRoom * widthRoom * heightRoom

### Environmental Parameters

In [46]:
pressure = 0.95 #atm, Used only for CO2 calculation
temperature = 20 # C, Use web converter if needed for F --> C. Used for CO2 calculation, eventually for survival rate of virus
relativeHumidity = 50 # percentage, Not yet used, but may eventually be used for survival rate of virus
backgroundCO2 = 415 # ppm 

### Occupants

In [47]:
numPeople = 31000

# N.B. Modified variable names to correspond with SIR models
I_0 = 1 # number of infected people at time 0
s_0 = 0 # fraction of people susceptible at time 0 
S_0 = (numPeople - I_0)*(1 - s_0) #number of susceptible people at time 0

# display(S_0)

density_AreaPerPerson = floorArea/(ft_to_meter**2)/numPeople #sq ft / person 
density_PeoplePerArea = numPeople/floorArea
density_VolumePerPerson = volumeRoom/numPeople

# display(density_AreaPerPerson)
# display(density_PeoplePerArea)
# display(density_VolumePerPerson)

#### Breath parameters

In [48]:
breathingRate = 0.012 * min_per_hour #See Readme sheet  - varies a lot with activity level
# as best as I can tell, this comes from a moderate intensity breath rate for people ages 16 - 21
# but I'm not really sure where this number comes from 
# Why is this value higher than the choir? 

basicBreathingRate = 0.0048 * min_per_hour # m3 / h, for a sedentary person in the 41 - 51 age groups (ASSUMPTION)
relBreathingRateFactor = breathingRate/basicBreathingRate

co2EmissionRate_person = 0.0061 # L/s (@ 273 K and 1 atm), From tables in Readme page. This does not affect infection calculation, only use of CO2 as indicator, could ignore
# could be completely incorrect for singing
co2EmissionRate_total = co2EmissionRate_person*numPeople*(1/pressure)*(C_to_Kelvin+temperature)/C_to_Kelvin # L/s (@ at actual P & T of room)

# display(relBreathingRateFactor)
# display(co2EmissionRate_total)

In [49]:
quantaExhalationRate = 18.6 # infectious doses (quanta) h-1, this is very uncertain
# The 5th and 95th percentiles of the Ep0 values are 8.4 and 48.1 quanta h−1, respectively.(Peng 2022)
variantEnhancement = 2.5 # Q. enhancement due to variants (omicron here)
vocalEnhancement = 25.0 # Q enhancement due to vocal. & act., Dimensionless (ratio to breathing)
# this value was not cited so I don't know where this came from

infectedQuantaExhalationRate = quantaExhalationRate * variantEnhancement * vocalEnhancement

# display(infectedQuantaExhalationRate)

##### Masking (none in this experiment)

In [50]:
exhalationMaskEfficiency = 0
propPeopleWithMasks = 0
inhalationMaskEfficiency = 0

### Event Parameters

In [51]:
eventDuration_mins = 90 # assumed for soccer game
eventDuration = eventDuration_mins/min_per_hour # hours
eventRepetitions = 1 # number of stadium events

# ventilation,  ventilation rate of 1 h-1 does not mean that 100% of the air is replaced in 1 h.
ventilationRate = 40 # h^-1, this may be low
virusDecayRate = 0.62 # h^-1, from literature. Uncertain parameter
depositionRate = 0.3 # h^-1,  Buonnano et al. (2020), Miller et al. (2020). Could vary 0.24-1.5 h-1, depending on particle size range
additionalControlMeasureRate = 0 # h^-1, E.g. filtering of recirc. air, HEPA air cleaner, UV disinfection, etc. 

# Loss Rate
firstOrderLossRate = ventilationRate + virusDecayRate + depositionRate + additionalControlMeasureRate

In [52]:
ventilationPerPerson = (volumeRoom * (ventilationRate + additionalControlMeasureRate)/numPeople) # [m^3/hour/person] 
ventilationPerPerson = ventilationPerPerson*L_per_m3/(sec_per_min * min_per_hour) # [L/s/person]

# display(ventilationPerPerson)

### Pathogen Parameters

For now, for COVID-19, Omicron variant

In [53]:
probInfective =0.002 # %, Very important parameter, specific for each region and time period. For ABSOLUTE results (prob. given prevalence of disease in the population). See Readme sheet
# this is highly dependent on phase of epidemic and region 

hospitalizationRate = 10/100
deathRate = 1/100

# for this example, this is probably an artificially tuned

## Model Outputs

### Main (calculate baseline emissions)

In [54]:
roomParams = lengthRoom, widthRoom, heightRoom
envParams = pressure, temperature, relativeHumidity, backgroundCO2 
occupants = numPeople, I_0, s_0
eventParams = eventDuration_mins, eventRepetitions
breathParams = breathingRate, basicBreathingRate, co2EmissionRate_person 
enhancements = quantaExhalationRate, variantEnhancement, vocalEnhancement
masking= exhalationMaskEfficiency, propPeopleWithMasks,inhalationMaskEfficiency
ventilation = ventilationRate, virusDecayRate, depositionRate, additionalControlMeasureRate 
pathogenParams = probInfective, hospitalizationRate, deathRate 

stadiumParams = [volumeRoom, numPeople, I_0, eventDuration_mins, eventRepetitions, propPeopleWithMasks, firstOrderLossRate]

netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam = main(roomParams, envParams, occupants, eventParams, breathParams, enhancements, masking, ventilation, pathogenParams)

stadiumRes = [netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam]
printTheResults(netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam)

The net emission rate is 1162.5
The average quanta concentration is 0.00010908372164737749
The quanta inhalation per person is 0.00011781041937916769
----------
The probability of infection is 0.00011780348000423047
The number of new cases is 3.65179007665114
The infection risk is 0.2727005070441173 [h^2*person/m^3]
The relative infection risk  is 8.79707432640141e-06[h^2/m^3]


# Comparison

In [55]:
from prettytable import PrettyTable

def printCompTable(choir, classroom, supermarket, subway, stadium):
    table = [['Example', 'Prob of Infection', 'Number of New Cases', 'Infection Risk', 'Relative Infection Risk']]
    tab = PrettyTable(table[0])
    tab.add_rows(table[1:])
    
    [netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam] = choir
    newRow = [['Choir', round(probOfInf,3), round(numNewCases,3), round(infRiskParam,3), round(relInfRiskParam,3) ]]
    tab.add_rows(newRow[0:])
    
    [netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam] = classroom
    newRow = [['Classroom', round(probOfInf,3), round(numNewCases,3), round(infRiskParam,3), round(relInfRiskParam,3) ]]
    tab.add_rows(newRow[0:])
    
    [netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam] = supermarket
    newRow = [['Supermarket', round(probOfInf,3), round(numNewCases,3), round(infRiskParam,3), round(relInfRiskParam,3) ]]
    tab.add_rows(newRow[0:])
    
    [netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam] = subway
    newRow = [['Subway', round(probOfInf,3), round(numNewCases,3), round(infRiskParam,3), round(relInfRiskParam,3) ]]
    tab.add_rows(newRow[0:])
    
    [netEmissionRate, avgQuantaConcentration, quantaInhaledPerPerson, probOfInf, numNewCases, infRiskParam, relInfRiskParam] = stadium
    newRow = [['Stadium', round(probOfInf,3), round(numNewCases,3), round(infRiskParam,3), round(relInfRiskParam,3) ]]
    tab.add_rows(newRow[0:])
        
    return tab
    

In [56]:
tab = printCompTable(choirRes, classRes, supermarketRes, subwayRes, stadiumRes)
print(tab)


+-------------+-------------------+---------------------+----------------+-------------------------+
|   Example   | Prob of Infection | Number of New Cases | Infection Risk | Relative Infection Risk |
+-------------+-------------------+---------------------+----------------+-------------------------+
|    Choir    |       0.888       |        53.275       |     24.513     |          0.409          |
|  Classroom  |       0.011       |        0.102        |     0.008      |          0.001          |
| Supermarket |       0.055       |        4.057        |     0.312      |          0.004          |
|    Subway   |       0.001       |        0.035        |     0.003      |           0.0           |
|   Stadium   |        0.0        |        3.652        |     0.273      |           0.0           |
+-------------+-------------------+---------------------+----------------+-------------------------+


In [57]:
tab = printCompTable(choirRes, classRes, supermarketRes, subwayRes, stadiumRes)

tab.add_column("Room Volume", [round(choirParams[0],3), round(classParams[0],3), round(supermarketParams[0],3), round(subwayParams[0],3), round(stadiumParams[0],3)])
tab.add_column("Number of People", [choirParams[1], classParams[1], supermarketParams[1], subwayParams[1], stadiumParams[1]])
tab.add_column("Event Duration (mins)", [choirParams[3], classParams[3], supermarketParams[3], subwayParams[3], stadiumParams[3]])
tab.add_column("First Order Loss Rate [h^-1]", [round(choirParams[6],3), round(classParams[6],3), round(supermarketParams[6],3), round(subwayParams[6],3), round(stadiumParams[6],3)])

print(tab)

+-------------+-------------------+---------------------+----------------+-------------------------+-------------+------------------+-----------------------+------------------------------+
|   Example   | Prob of Infection | Number of New Cases | Infection Risk | Relative Infection Risk | Room Volume | Number of People | Event Duration (mins) | First Order Loss Rate [h^-1] |
+-------------+-------------------+---------------------+----------------+-------------------------+-------------+------------------+-----------------------+------------------------------+
|    Choir    |       0.888       |        53.275       |     24.513     |          0.409          |   808.128   |        61        |          150          |             1.62             |
|  Classroom  |       0.011       |        0.102        |     0.008      |          0.001          |   143.716   |        10        |           50          |             3.92             |
| Supermarket |       0.055       |        4.057       